Implementation of Fake News Detection Model

1.Importing Libraries
numpy, pandas, scikit learn and tenserflow

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2. Importing the Dataset

In [3]:
data = pd.read_csv("news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


3. Preprocessing Dataset

to drop that column from the dataset.

In [4]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


4. Data Encoding

to convert numerical values


le.fit(data['label']): Fits the encoder on the 'label' column to learn the unique categories.

data['label'] = le.transform(data['label']): Transforms the categorical labels into numerical format (0 for REAL, 1 for FAKE).

In [5]:
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

5. Variables Setup

These are some variables required to be setup for the model training.

In [6]:
embedding_dim = 50
max_length = 54
padding_type = 'post'
trunc_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = 0.1

6. Tokenization

divides a large piece of continuous text into distinct units or tokens.
Here we use columns separately for a temporal basis as a pipeline just for good accuracy.

tokenizer1.fit_on_texts(title): Fits the tokenizer on the 'title' column to create a vocabulary.
pad_sequences(sequences1): Pads the sequences to ensure they all have the same len
gth.

In [7]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(sequences1, padding=padding_type, truncating=trunc_type)

7. Splitting Data for Training and Testing


training_sequences1, test_sequences1: Splits the tokenized and padded data into training and testing sets.

training_labels, test_labels: Splits the corresponding labels into training and testing labels.


In [8]:
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

8. Reshaping Data for LSTM

using LSTM(Long Short Term Memory) model for prediction and for that we need to reshape padded sequence. We are converting it into np.array() as we need training and test sequences into NumPy arrays which are required by TensorFlow models.

In [9]:
training_sequences1 = np.array(training_sequences1)
test_sequences1 = np.array(test_sequences1)

9. Generating Word Embedding

Embeddings allows words with similar meanings to have a similar representation. Here each individual word is represented as real-valued vectors in a predefined vector space. For that we will be using glove.6B.50d.txt.

!wget: Downloads the pre-trained GloVe embeddings from the following link.

!unzip: Unzips the downloaded file containing the GloVe embeddings.

In [10]:
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

--2025-10-20 09:20:48--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.00MB/s    in 2m 39s  

2025-10-20 09:23:27 (5.17 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


our glove embeddings are downloaded we can use them for word embedding

In [11]:
embedding_index = {}
with open('glove.6B.50d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size1 + 1, embedding_dim))

for word, i in word_index1.items():
    if i < vocab_size1:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

10. Model Architecture

**we** use the TensorFlow embedding technique with Keras Embedding Layer where we map original input data into some set of real-valued dimensions.

Embedding: The embedding layer uses pre-trained GloVe embeddings.

Conv1D: A 1D convolutional layer to detect patterns in the text.

LSTM(64): An LSTM layer to capture long-term dependencies in the data.

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1 + 1, embedding_dim, input_length=max_length,
                              weights=[embedding_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       377,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 377,600 (1.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 377,600 (1.44 MB)

11. Training the Model

In [13]:
history = model.fit(
    training_sequences1,
    np.array(training_labels),
    epochs=50,
    validation_data=(test_sequences1, np.array(test_labels)),
    verbose=2
)

Epoch 1/50
85/85 - 5s - 62ms/step - accuracy: 0.6381 - loss: 0.6378 - val_accuracy: 0.6600 - val_loss: 0.6105
Epoch 2/50
85/85 - 2s - 23ms/step - accuracy: 0.6837 - loss: 0.5852 - val_accuracy: 0.6733 - val_loss: 0.5620
Epoch 3/50
85/85 - 1s - 15ms/step - accuracy: 0.7196 - loss: 0.5469 - val_accuracy: 0.6800 - val_loss: 0.5791
Epoch 4/50
85/85 - 1s - 16ms/step - accuracy: 0.7626 - loss: 0.4986 - val_accuracy: 0.7300 - val_loss: 0.5239
Epoch 5/50
85/85 - 1s - 17ms/step - accuracy: 0.7770 - loss: 0.4625 - val_accuracy: 0.7533 - val_loss: 0.4974
Epoch 6/50
85/85 - 2s - 28ms/step - accuracy: 0.8337 - loss: 0.3903 - val_accuracy: 0.7100 - val_loss: 0.5273
Epoch 7/50
85/85 - 2s - 29ms/step - accuracy: 0.8430 - loss: 0.3612 - val_accuracy: 0.7000 - val_loss: 0.5856
Epoch 8/50
85/85 - 1s - 17ms/step - accuracy: 0.8467 - loss: 0.3484 - val_accuracy: 0.7433 - val_loss: 0.5255
Epoch 9/50
85/85 - 2s - 29ms/step - accuracy: 0.8715 - loss: 0.3076 - val_accuracy: 0.7433 - val_loss: 0.5584
Epoch 10/5

For each epoch training accuracy improves reaching around 98% by the 50th epoch while the validation accuracy is around 75%.

12. Sample Prediction

In [25]:
#X = "San goes to myanmar"
X = "Karry to go to France in gesture of sympathy"

sequences = tokenizer1.texts_to_sequences([X])
sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
if model.predict(sequences, verbose=0)[0][0] >= 0.5:
    print("This news is True")
else:
    print("This news is False")

This news is True
